# Notebook for 2022 Feb Qiskit Metal Study

Gyeonghun Kim (Qiskit Advocate / Seoul National University)

This jupyter notebook is created for 2022-Feb Qiskit Metal Study organized by Qiskit Korea and AiQyAm. This notebook covers basics about creating, copying and changing parameters for TransmonPockets and Transmission lines. To get more detailed knowledge, please below reference. 

Reference: 
https://github.com/Qiskit/qiskit-metal/tree/main/tutorials/2%20From%20components%20to%20chip

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

%metal_heading Welcome to Qiskit Metal Study!

## 1. Setup Planar design and GUI

In [ ]:
# To start qiskit-metal, you have to create instance of below two.
# After runing this cell, metal gui will appear.
design = designs.DesignPlanar()
gui = MetalGUI(design)

In [ ]:
# If you delete all components in the current design, you can use delete_all_components method.
design.delete_all_components()

In [ ]:
# To apply changes in design instance to your gui, you can use rebuild method in gui.
gui.rebuild()

## 2. TransmonPocket

Below, I will only show examples for TransmonPocket. However, since the syntax is equivalant for all other qcomponents, you can easily adopt this to other.

### 2.1. Add Transmon qubit

In [ ]:
# You can import templated qubit class from qiskit_metal.qlibrary.qubits.
# For transmon pocket, 
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket


In [ ]:
# For most simplest case, you can create TransmonPocket instance with default options as below.
# Methods for creating QComponent with options will be explained in the section 2.4
# Please ignore warnings here. These are caused by Shapely's future update, and would not make any troubles now.
q1 = TransmonPocket(design, 'Q1')
gui.rebuild()

In [ ]:
# If you call TransmonPocket instance, you can see the parameters of it. 
q1

In [ ]:
# Above options are same with TransmonPocket's default option
TransmonPocket.default_options

### 2.2. Change parameters 

In [ ]:
# If you want to change parameters of Transmon qubit, 
# you can simply change the parameters in the q1.options as below
q1.options.pos_x = '2.0 mm' # Note that if you don't input the unit here, default length unit is mm not um.
q1.options.pos_y = '2.0 mm'
q1.options.pad_height = '250 um'
q1.options.pad_width  = '300 um'

In [ ]:
# By rebuilding gui, you can check the parameters are changed.
gui.rebuild()

### 2.3. Copy QComponent

In [ ]:
# In many cases, copying QComponents is very useful. 
# To copy single QComponent, you can use copy_qcomponent method for design instance.
q2 = design.copy_qcomponent(q1, 'Q2')
# If you want to change parameters for copyed object, you can do it with two different way. 
# First method is same with above explained changing parameters.
q2.options.pos_x = '-2.0mm'
gui.rebuild()
# By runing below, you can automatly scale the gui.
gui.autoscale()

In [ ]:
# Second method is giving options to the copy_qcomponent as below.
q3 = design.copy_qcomponent(q1, 'Q3', dict(pos_x='-4.0mm'))

In [ ]:
# If you want to copy multiple qcomponent, you can use below method. 
newcopies = design.copy_multiple_qcomponents([q1, q2], ['Q3', 'Q4'], [dict(pos_y='-2.0mm'), dict(pos_y='-2.0mm')])
gui.rebuild()
gui.autoscale()

### 2.4. Add connecting pads 

Qubit have to be connected with transmission line for operation, readout, and connection with other qubits. Transmission lines are coupled with superconducting qubit, therefore, we have to create connecting pads in the transmon qubit to connect with the transmission line.

In [ ]:
design.delete_all_components()
gui.rebuild()

In [ ]:
# To create Transmon Qubit with option, the only thing you have to do is 
# creating dictionary of options and give it as a input when you create TransmonPocket instance.
# You can give options directly, without create dictionary as variable, like below, 

# Adding connection pads also can be done by add "connection_pads" in your option dictionary as below.
q1 = TransmonPocket(design, 'Q1', 
                    options = dict(
                        pos_x='-1.5mm', 
                        pos_y='+0.0mm', 
                        pad_width = '425 um',
                        pocket_height = '650um',
                        connection_pads = dict(
                            a = dict(loc_W=+1,loc_H=+1), # (low_W, low_H) = (1, 1) => First Quadrant
                            b = dict(loc_W=-1,loc_H=+1, pad_height='30um'), # (low_W, low_H) = (-1, 1) => Second Quadrant
                            c = dict(loc_W=+1,loc_H=-1, pad_width='200um'), # (low_W, low_H) = (1, -1) => Fourth Quadrant
                            d = dict(loc_W=-1,loc_H=-1, pad_height='50um') # (low_W, low_H) = (-1, -1) => Third Quadrant
                        ) # Note that connection_pads have two geometrical options pad_height and pad_weight.
                          # You can modify those values from default, by add them in option as above.
                    )
                )
gui.rebuild()


In [ ]:
design.delete_all_components()
gui.rebuild()

In [ ]:
# I prefer below method, which is very convenient for reusing the parameters.

optionsQ = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads = dict(  # Qbits defined to have 4 pins
        a = dict(loc_W=+1,loc_H=+1), 
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)


q1 = TransmonPocket(design, 'Q1', options = dict(pos_x='-1.5mm', pos_y='+0.0mm', **optionsQ))
q2 = TransmonPocket(design, 'Q2', options = dict(pos_x='+0.35mm', pos_y='+1.0mm', orientation = '90',**optionsQ))
q3 = TransmonPocket(design, 'Q3', options = dict(pos_x='2.0mm', pos_y='+0.0mm', **optionsQ))

gui.rebuild()
gui.autoscale()
gui.highlight_components(['Q1', 'Q2', 'Q3'])  # This is to show the pins, so we can choose what to connect
gui.rebuild()
gui.autoscale()

## 3. Transmission line

Since length of the transmission line is key factor of determining resonant frequency. However, since the chip size is restricted, we have to make meander transmission lines.

In [ ]:
# You can import RouteMeander class as below
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [ ]:
# Same with TransmonPocket, it also have its default options
RouteMeander.get_template_options(design)

### 3.1. Connecting two superconducting qubit

In [ ]:
# Connecting two TransmonPocket with RouteMinder is very easy.
# It can be done by setting start_pin and end_pin in option as below
ops=dict(fillet='90um')
options = Dict(
    total_length= '8mm', # This parameter is key parameter for changing resonant frequency
    hfss_wire_bonds = True,
    pin_inputs=Dict(
        start_pin=Dict(
            component= 'Q1', # Start component Name
            pin= 'a'), # Start Pin in component
        end_pin=Dict(
            component= 'Q2',
            pin= 'b')),
    lead=Dict(
        start_straight='0mm', # How long go straight from the starting point
        end_straight='0.5mm'), # How long go straight to the end point
    meander=Dict(
       asymmetry='-1mm'),
    **ops # Remark that this syntax usually used for reusing options.
)

cpw = RouteMeander(design, options=options)
gui.rebuild()
gui.autoscale()

If you check gui after run above code, you can see the line pass not expected pin. To solve that issue, you can increase "start_straight" value as below. Syntax for changing parameters in transmission line is same with the Transmon pocket.

In [ ]:
cpw.options.lead.start_straight='100um'
gui.rebuild()
gui.autoscale()

Fillet in option is about radius in corner of transmission line. Sometimes, transmission lines have angulate. In this case, you can make this rough by reducing fillet radius as below.

In [ ]:
cpw.options['fillet']='30um'
gui.rebuild()
gui.autoscale()

### 3.2. Custom jogs

You can use custom jogs in your transmission line by add start_jogged_extension or end_jogged_extansion in your option.

In [ ]:
from collections import OrderedDict
jogs = OrderedDict()
jogs[0] = ["L", '800um']
jogs[1] = ["L", '500um']
jogs[2] = ["R", '200um']
jogs[3] = ["R", '500um']
 
options = Dict(
    total_length= '14mm',
    pin_inputs=Dict(
        start_pin=Dict(
            component= 'Q1',
            pin= 'd'),
        end_pin=Dict(
            component= 'Q3',
            pin= 'd')),
    lead=Dict(
        start_straight='0.1mm',
        end_straight='0.1mm',
        start_jogged_extension=jogs,
        end_jogged_extension=jogs),
    meander=Dict(
       asymmetry='-1.2mm'),
    **ops
)

try:
    cpw5.delete()
except NameError: pass

cpw5 = RouteMeander(design,options=options)
gui.rebuild()
gui.autoscale()

### 3.3. Anchors

Another way to force the path of the transmission line is creating anchors. This also can be done by using RoutePathfinder and adding anchors in your options. 

In [ ]:
# Before showing examples of anchors, I will reset design and add some new transmon pockets. 

design.delete_all_components()
gui.rebuild()

options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(  # pin connectors
        a = dict(loc_W=+1,loc_H=+1), 
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

q0 = TransmonPocket(design, 'Q0', options = dict(pos_x='-1.0mm', pos_y='-1.0mm', **options))
q1 = TransmonPocket(design, 'Q1', options = dict(pos_x='1.0mm', pos_y='+0.0mm', **options))

gui.rebuild()
gui.autoscale()

In [ ]:
# You can import RoutePathfinder as below
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from collections import OrderedDict
# You should add anchors in OrderedDict because the order in anchors are very important.
anchors = OrderedDict()
anchors[0] = np.array([-0.452, -0.555])
anchors[1] = np.array([-0.452, -1.5])
anchors[2] = np.array([0.048, -1.5])

options = {'pin_inputs': 
            {'start_pin': {'component': 'Q0', 'pin': 'b'}, 
             'end_pin': {'component': 'Q1', 'pin': 'b'}},
            'lead': {'start_straight': '91um', 'end_straight': '95um'},
            'step_size': '0.25mm',
            'anchors': anchors,
            **ops
           }

qa = RoutePathfinder(design, 'line', options)

gui.rebuild()
gui.autoscale()